In [1]:
#!/usr/bin/env python
import sys
if sys.version_info[0] >= 3:
    import PySimpleGUI as sg
else:
    import PySimpleGUI27 as sg
import cv2
from PIL import Image
import io
from sys import exit as exit
import numpy as np
import pafy
import re

cap = cv2.VideoCapture(0) # Get input from webcam

# Data for categorising age predictions
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

def initialize_caffe_models():
    #load .prototxt and caffemodel for both gender and Age
    age_net = cv2.dnn.readNetFromCaffe('deploy_age.prototxt', 'age_net.caffemodel') 

    gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')

    return(age_net, gender_net)

def main(age_net, gender_net):
    
    font = cv2.FONT_HERSHEY_SIMPLEX

    sg.ChangeLookAndFeel('BlueMono') # Colour scheme for self service checkout

    # Window layout
    layout = [[sg.Text('Self-Service Checkout', size=(40, 1), justification='center', font='Arial 20')],
              [sg.Image(filename='', key='image')],
              [sg.ReadButton('Exit', size=(10, 1), pad=((200, 0), 3), font='Helvetica 14'),
               sg.RButton('Alcohol', size=(10,1), font='Any 14'), 
               sg.RButton('Bread', size=(10,1), font='Any 14')]]

    # Create and show the window
    window = sg.Window('Self-Service Checkout',
                       location=(800,400))
    window.Layout(layout).Finalize()
    
    # Loop read and display frames and assign jobs to buttons
    while True:
        
        button, values = window.read(timeout = 100)

        if button is 'Exit' or values is None:
            sys.exit(0)
        elif button == 'Alcohol':
            age_verification()
        elif button == 'Bread':
            sg.PopupNoWait('No age verification required.',
                           'You can proceed to purchase your bread.',
                           keep_on_top=True)

        ret, frame = cap.read() # Read data from webcam
        
        # Load pre built model for facial recognition
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        
        if(len(faces)>0):
            print("Found {} faces".format(str(len(faces))))

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

            # Get Face 
            face_img = frame[y:y+h, h:h+w].copy()
            blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # Predict Gender
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_list[gender_preds[0].argmax()]
            print("Gender : " + gender)

            # Predict Age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_list[age_preds[0].argmax()]
            print("Age Range: " + age)
            
            # Set minimum age for age verficiation method
            if age == '(0, 2)':
                verifAge = 0
            elif age == '(4, 6)':
                verifAge = 4
            elif age == '(8, 12)':
                verifAge = 8
            elif age == '(15, 20)':
                verifAge = 15
            elif age == '(25, 32)':
                verifAge = 25
            elif age == '(38, 43)':
                verifAge = 38
            elif age == '(48, 53)':
                verifAge = 48
            elif age == '(60, 100)':
                verifAge = 60

            overlay_text = "%s %s" % (gender, age)
            cv2.putText(frame, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.imshow('AI View', frame)

        # let img be the PIL image
        img = Image.fromarray(gray)  # create PIL image from frame
        bio = io.BytesIO()  # a binary memory resident stream
        img.save(bio, format= 'PNG')  # save image as png to it
        imgbytes = bio.getvalue()  # this can be used by OpenCV hopefully
        window.FindElement('image').Update(data=imgbytes)
        
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
        
        # Method to output whether user can buy restricted items
        def age_verification():
            if verifAge >= 25:
                sg.PopupNoWait('Age verification test!',
                'You have passed the age verification test.',
                'You may proceed to purchase your alcohol.',
                keep_on_top=True)
            else:
                sg.PopupNoWait('Age verification test!',
                'You have failed the age verification test.',
                'Please wait to have a manual ID check by a store employee.',
                'An employee will be with you shortly.',
                keep_on_top=True)

if __name__ == "__main__":
    age_net, gender_net = initialize_caffe_models()

    main(age_net, gender_net)

Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Foun

Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (

Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Ran

Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Female
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gend

Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (38, 43)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age 

Found 1 faces
Gender : Male
Age Range: (8, 12)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (0, 2)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (8, 12)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (60, 100)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (8, 12)
Found 1 faces
Gender : Male
Age Range: (8, 12)
Found 2 faces
Gender : Female
Age Range: (48, 53)
Gender : Female
Age Range: (25, 32)
Found 1 faces
Gender : Female
Age Range: (25, 32)
Found 2 faces
Gender : Female
Age Range: (25, 32)
Gender : Female
Age Range: (25, 32)
Found 2 faces
Gender : Female
Age Range: (25, 32)
Gender :

Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Female
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (4, 6)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 2 faces
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (25, 32)
Found 2 faces
Gender : Female
Age Range: (8, 12)
Gender : Male
Age Range: (48, 53)
Found 2 faces
Gender : Female
Age Range: (4, 6)
Gender : Male
Age Range: (48, 53)
Found 1 faces
Gender : Male
Age Range: (48, 53)
Found 2 faces
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (25, 32)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 2 faces
Gender : Male
Age Range: (25, 32)
Gender : Female
Age Range: (15, 20)
Found 2 faces
Gender : Male
Age Range: (25, 32)
Gender : Male
Age Range: (15, 20)
Found 2 faces
Gender : Female
Age Range: (15, 20)
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (25, 32)
Found 2 faces
Gender : Female
Age Range: (25, 32)
Gender : Male
Age Range: (15, 20)
Fou

Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 faces
Gender : Male
Age Range: (15, 20)
Found 1 faces
Gender : Female
Age Range: (15, 20)
Found 1 fa

SystemExit: 0

C:\Users\melis\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
